# Merging Datasets
# 02_EDA_merging_files

| Date | User | Change Type | Remarks |  
| ---- | ---- | ----------- | ------- |
| 22/09/25 | Adrienne | Uppdate | Made significant changes to program | 
|    | |   | |

### Content

* [Introduction](#introduction)

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from datetime import datetime
import json_lines

In [25]:
# readin clean datafiles
path = "../data/clean/"
eob_df = pd.read_pickle(path + 'explanation_of_benefit_clean.pkl')
coverage_df = pd.read_pickle(path + 'coverage.pkl')
claim_df =  pd.read_pickle(path + 'claim.pkl')
claim_response_df =  pd.read_pickle(path + 'claim_response.pkl')
patient_df = pd.read_pickle(path + 'patient.pkl')

Creating some common variables across datafiles

In [67]:
# Preprocess identifiers in EOB
#eob_df['patient_medicare_number'] = eob_df['contained_0_identifier_0_value']
eob_df['patient_number'] = eob_df['id'].str.replace(r'[-inpatientout]', '', regex=True)
eob_df['unique_claim_ID'] = eob_df['claimId_2'].str.replace(r'[-]', '', regex=True)
#eob_df['hcpcs_code'] = eob_df['item_0_productOrService_coding_0_code']

In [19]:
eob_df.head()

,careTeam,created,diagnosis,extension,id,insurance,insurer,item,meta,outcome,...,paymentAmount,paymentDate,providerId,totalChargeType,totalChargeCurrency,totalChargeAmount,claimType,eobType,patient_number,Unique Claim ID
0,[{'provider': {'identifier': {'type': {'coding...,2025-08-31T22:12:12+00:00,[{'diagnosisCodeableConcept': {'coding': [{'co...,[{'url': 'https://bluebutton.cms.gov/resources...,10000002646806,[{'coverage': {'reference': 'Coverage/part-a--...,CMS,[{'adjudication': [{'amount': {'currency': 'US...,{'lastUpdated': '2021-08-17T17:47:01.129+00:00...,complete,...,129.16,NaN,NaN,Claim Total Charge Amount,USD,129.16,institutional,INPATIENT,10000002646806,102852906
1,[{'provider': {'identifier': {'type': {'coding...,2025-08-31T22:12:12+00:00,[{'diagnosisCodeableConcept': {'coding': [{'co...,[{'url': 'https://bluebutton.cms.gov/resources...,10000002646833,[{'coverage': {'reference': 'Coverage/part-a--...,CMS,[{'adjudication': [{'amount': {'currency': 'US...,{'lastUpdated': '2021-08-17T17:47:01.129+00:00...,complete,...,134.47,NaN,NaN,Claim Total Charge Amount,USD,134.47,institutional,INPATIENT,10000002646833,102852934
2,[{'provider': {'identifier': {'type': {'coding...,2025-08-31T22:12:12+00:00,NaN,[{'url': 'https://bluebutton.cms.gov/resources...,10000002646839,[{'coverage': {'reference': 'Coverage/part-b--...,CMS,[{'adjudication': [{'amount': {'currency': 'US...,{'lastUpdated': '2021-08-17T17:48:33.131+00:00...,complete,...,218.01,NaN,NaN,Claim Total Charge Amount,USD,218.01,institutional,OUTPATIENT,10000002646839,102852942
3,[{'provider': {'identifier': {'type': {'coding...,2025-08-31T22:12:12+00:00,NaN,[{'url': 'https://bluebutton.cms.gov/resources...,10000002646843,[{'coverage': {'reference': 'Coverage/part-b--...,CMS,[{'adjudication': [{'amount': {'currency': 'US...,{'lastUpdated': '2021-08-17T17:48:33.131+00:00...,complete,...,218.01,NaN,NaN,Claim Total Charge Amount,USD,218.01,institutional,OUTPATIENT,10000002646843,102852947
4,[{'provider': {'identifier': {'type': {'coding...,2025-08-31T22:12:12+00:00,[{'diagnosisCodeableConcept': {'coding': [{'co...,[{'url': 'https://bluebutton.cms.gov/resources...,10000002646848,[{'coverage': {'reference': 'Coverage/part-b--...,CMS,[{'adjudication': [{'amount': {'currency': 'US...,{'lastUpdated': '2021-08-17T17:48:33.131+00:00...,complete,...,218.01,NaN,NaN,Claim Total Charge Amount,USD,218.01,institutional,OUTPATIENT,10000002646848,102852953


In [6]:
# It seems that eob data does not have patient medicare number, but intsead just patient number
print(f"EOB columns: {eob_df.columns}")
# patient number does not seem to be unique 
#eob_df.patient.describe()

EOB columns: Index(['careTeam', 'created', 'diagnosis', 'extension', 'id', 'insurance',
       'insurer', 'item', 'meta', 'outcome', 'patient', 'resourceType',
       'status', 'subType', 'supportingInfo', 'use', 'disposition',
       'procedure', 'benefitCategory', 'benefitFinancials',
       'billablePeriodStart', 'billablePeriodEnd', 'ClaimType', 'active',
       'PRNCode', 'NPICode', 'facilityId', 'faciltyType', 'claimId_1',
       'claimId_2', 'coveragePart', 'coverageId', 'lastUpdated',
       'paymentCurrency', 'paymentAmount', 'paymentDate', 'providerId',
       'totalChargeType', 'totalChargeCurrency', 'totalChargeAmount',
       'claimType', 'eobType'],
      dtype='object')


In [28]:
# Preprocess Identifier in Patient
patient_df['patient_medicare_number'] = patient_df['identifier_1_value']
patient_df['patient_number'] = patient_df['identifier_0_value'].str.replace(r'[-]', '', regex=True)
patient_df['patient_first_name'] = patient_df['name_0_given'].str.replace(r'[ \[ \]"]', '', regex=True)
patient_df['patient_last_name'] = patient_df['name_0_family']

In [70]:
patient_df.head()

,address_0_state,birthDate,deceasedBoolean,extension_0_url,extension_0_valueCode,extension_1_url,extension_1_valueCoding_code,extension_1_valueCoding_display,extension_1_valueCoding_system,extension_2_extension_0_url,...,identifier_2_type_coding_0_extension_0_url,identifier_2_type_coding_0_extension_0_valueCoding_code,identifier_2_type_coding_0_extension_0_valueCoding_display,identifier_2_type_coding_0_extension_0_valueCoding_system,identifier_2_type_coding_0_system,identifier_2_value,patient_medicare_number,patient_number,patient_first_name,patient_last_name
0,22,1953-10-12,0.0,http://hl7.org/fhir/us/core/StructureDefinitio...,248152002,https://bluebutton.cms.gov/resources/variables...,1,White,https://bluebutton.cms.gov/resources/variables...,ombCategory,...,NaN,NaN,NaN,NaN,NaN,NaN,1S00E00FR92,10000000009392,Mina319,Mohr916
1,22,1946-03-01,0.0,http://hl7.org/fhir/us/core/StructureDefinitio...,248152002,https://bluebutton.cms.gov/resources/variables...,1,White,https://bluebutton.cms.gov/resources/variables...,ombCategory,...,NaN,NaN,NaN,NaN,NaN,NaN,1S00E00AK52,10000000000852,Leda374,Kshlerin58
2,22,1947-08-31,0.0,http://hl7.org/fhir/us/core/StructureDefinitio...,248153007,https://bluebutton.cms.gov/resources/variables...,2,Black,https://bluebutton.cms.gov/resources/variables...,ombCategory,...,NaN,NaN,NaN,NaN,NaN,NaN,1S00E00FU79,10000000009579,Jeromy156,Morissette863
3,22,1952-01-13,0.0,http://hl7.org/fhir/us/core/StructureDefinitio...,248153007,https://bluebutton.cms.gov/resources/variables...,5,Hispanic,https://bluebutton.cms.gov/resources/variables...,ombCategory,...,NaN,NaN,NaN,NaN,NaN,NaN,1S00E00CY77,10000000003977,Sergio619,Vanegas191
4,22,1943-10-05,0.0,http://hl7.org/fhir/us/core/StructureDefinitio...,248153007,https://bluebutton.cms.gov/resources/variables...,1,White,https://bluebutton.cms.gov/resources/variables...,ombCategory,...,NaN,NaN,NaN,NaN,NaN,NaN,1S00E00AY95,10000000001995,Fernando603,Aufderhar910


In [30]:
# EOB does not have patient medicare number so will use patient number
eob_df_part = eob_df[['patient_number','unique_claim_ID' ]]
patient_df_part = patient_df[['patient_number', 'patient_medicare_number']]
claim_df_part = claim_df[['patient_medicare_number', 'unique_claim_ID']]
claim_response_df_part = claim_response_df[['patient_medicare_number', 'unique_claim_ID']]


In [41]:
print(f"num of unique patient numbers in EOB: {len(eob_df['patient_number'].unique())}")
print(f"num of unique patient numbers in Patient: {len(patient_df['patient_number'].unique())}")
print(f"num of unique patient medicare numbers in Patient: {len(patient_df['patient_medicare_number'].unique())}")
print(f"num of unique patient medicare numbers in Claim: {len(claim_df['patient_medicare_number'].unique())}")
print(f"num of unique patient medicare numbers in Claim Response: {len(claim_response_df['patient_medicare_number'].unique())}")

num of unique patient numbers in EOB: 4091
num of unique patient numbers in Patient: 5000
num of unique patient medicare numbers in Patient: 5000
num of unique patient medicare numbers in Claim: 15
num of unique patient medicare numbers in Claim Response: 2872


In [68]:
# Checking lengths of values in columns
print(eob_df['patient_number'].str.len().unique())
print(patient_df['patient_number'].str.len().unique())
print(patient_df['patient_medicare_number'].str.len().unique())
print(claim_response_df['patient_medicare_number'].str.len().unique())

[14 11]
[14]
[11]
[11]


In [69]:
# EOB diagnosis 21
# Claim diagnossi 23

Merging files together to see if most records are found in each file

In [35]:
df = pd.merge(eob_df_part, patient_df_part, how = 'outer', on = 'patient_number')


In [38]:
df.describe()

,patient_number,unique_claim_ID,patient_medicare_number
count,9091,4091,5000
unique,9091,4091,5000
top,10000000000000,100015890,1S00E00FZ00
freq,1,1,1


In [40]:
df.patient_medicare_number.isnull().sum()

np.int64(4091)

In [42]:
df.tail(100)

,patient_number,unique_claim_ID,patient_medicare_number
8991,10000215305,102964775,NaN
8992,10000215306,102964778,NaN
8993,10000215307,102964781,NaN
8994,10000215308,102964783,NaN
8995,10000215309,102964788,NaN
...,...,...,...
9086,10000231993,103213538,NaN
9087,10000231994,103213540,NaN
9088,10000231995,103213542,NaN
9089,10000231996,103213545,NaN


In [ ]:
df1 = pd.merge(claim_response_df_part, patient_df_part, how = 'outer', on = 'patient_medicare_number')

In [53]:
df1.describe()

,patient_medicare_number,unique_claim_ID,patient_number
count,183629,178761,22797
unique,7740,170787,5000
top,1S00E00GT33,100085222,10000000000811
freq,2550,3,499


In [54]:
df1.patient_number.isnull().sum()

np.int64(160832)

In [58]:
df1.tail()

,patient_medicare_number,unique_claim_ID,patient_number
183624,1S00E00MY91,100518239,NaN
183625,1S00E00MY91,100518241,NaN
183626,1S00E00MY91,100518243,NaN
183627,1S00E00MY91,100518244,NaN
183628,1S00E00MY91,100518245,NaN
